Basic Model (no image preprocessing & no data augmentation)

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow import keras

In [2]:
train_dir = r'C:\Users\geetika singh\Downloads\Project - FaceMask Detection\FaceMaskDataset\Train'
validation_dir = r'C:\Users\geetika singh\Downloads\Project - FaceMask Detection\FaceMaskDataset\Validation'
test_dir = r'C:\Users\geetika singh\Downloads\Project - FaceMask Detection\FaceMaskDataset\Test'

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(128, 128),
        batch_size=20,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_dir,
        target_size=(128, 128),
        batch_size=20,
        class_mode='binary')

test_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary')

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


In [5]:
from keras import layers, models, optimizers

In [6]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(512, activation='relu'))

model.add(layers.Dense(1, activation='sigmoid'))

In [7]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

In [8]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [9]:
checkpoint_cb1 = ModelCheckpoint("FaceMask_Sans_Data_Augm.h5", save_best_only=True)
early_stopping_cb1 = EarlyStopping(patience=3, restore_best_weights=True)

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=500,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=40, 
    callbacks=[checkpoint_cb1, early_stopping_cb1])

In [10]:
model = keras.models.load_model("FaceMask_Sans_Data_Augm.h5")

In [11]:
model.evaluate(test_generator, steps=31)

31/31 [==============================] - 15s 482ms/step - loss: 0.0245 - acc: 0.9899


[0.024493617936968803, 0.9899193644523621]

In [ ]:
keras.backend.clear_session()
del model

## Data Augmentation

In [14]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary')

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

In [ ]:
checkpoint_cb2 = ModelCheckpoint("FaceMask_with_Data_Augm.h5", save_best_only=True)
early_stopping_cb2 = EarlyStopping(patience=3, restore_best_weights=True)

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=300,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=25, 
    callbacks=[checkpoint_cb2, early_stopping_cb2])

In [12]:
model = keras.models.load_model("FaceMask_with_Data_Augm.h5")
model.evaluate(test_generator)

31/31 [==============================] - 6s 182ms/step - loss: 0.0854 - acc: 0.9748


[0.08543869107961655, 0.9747983813285828]

In [13]:
keras.backend.clear_session()
del model

## VGG19

In [15]:
conv_base = keras.applications.VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(128, 128, 3))

In [16]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [17]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=2e-5),
              metrics=['acc'])

In [18]:
checkpoint_cb3 = ModelCheckpoint("FaceMask_VGG19.h5", save_best_only=True)
early_stopping_cb3 = EarlyStopping(patience=3, restore_best_weights=True)

In [ ]:
history = model.fit(train_generator, epochs=10,
      validation_data=validation_generator,
      callbacks=[checkpoint_cb3, early_stopping_cb3])

In [ ]:
model = keras.models.load_model("FaceMask_VGG19.h5")
model.evaluate(test_generator)